In [1]:
import boto3
import awswrangler as wr
import pandas as pd
import csv
import json
import warnings 
import logging
import tempfile
import requests

from botocore.exceptions import ClientError
warnings.filterwarnings('ignore')

import configparser

config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))


KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')
REGION                 = "us-west-2"

In [2]:
import boto3
import zipfile
import tempfile
import os

# AWS credentials and region
aws_access_key_id = KEY
aws_secret_access_key = SECRET
region_name = REGION

# Lambda function details
lambda_function_name = 'glue_dynamo_trigger'
handler_name = 'lambda_function.handler'  # Replace 'lambda_function' with your module and 'handler' with your function
runtime = 'python3.8'
role_arn = 'arn:aws:iam::935670829844:role/GlueCrawlerRole'  # Replace with the ARN of your existing IAM role

# Python script content for the Lambda function
python_script_content = """
def handler(event, context):
    return {
        'statusCode': 200,
        'body': 'Hello from Lambda!'
    }
"""

# Create a temporary zip file
temp_zip_file = tempfile.mktemp()
with zipfile.ZipFile(temp_zip_file, 'w') as zip_file:
    zip_file.writestr('lambda_function.py', python_script_content)

# Create a Lambda client
lambda_client = boto3.client('lambda', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key, region_name=region_name)

# Upload the zip file to Lambda
with open(temp_zip_file, 'rb') as zip_file:
    zip_file_content = zip_file.read()

try:
    response = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime=runtime,
        Role=role_arn,
        Handler=handler_name,
        Code={
            'ZipFile': zip_file_content
        }
    )
    lambda_function_arn = response['FunctionArn']
    print("Lambda Function ARN:", lambda_function_arn)
except Exception as e:
    print(f"Error creating Lambda function: {e}")

# Clean up: Delete the temporary zip file
os.remove(temp_zip_file)


Lambda Function ARN: arn:aws:lambda:us-west-2:935670829844:function:glue_dynamo_trigger


In [ ]:
'''
Check S3 Bucket Configuration:
S3 Bucket Event Notification:
Navigate to the S3 service in the AWS Management Console.
Find your S3 bucket and open its properties.
Go to the "Event notifications" tab and check the Lambda function configuration.
'''

In [14]:
import subprocess
import json

# Replace these variables with your own values
bucket_name = 'ieshaan-bucket-latest'
lambda_function_arn = 'arn:aws:lambda:us-west-2:935670829844:function:glue_dynamo_trigger'
region_name = 'us-west-2'  # Replace with your actual AWS region

# Create an S3 client
s3_client = boto3.client('s3', aws_access_key_id=KEY, aws_secret_access_key=SECRET, region_name=region_name)

# Define the notification configuration
notification_config = {
    'LambdaFunctionConfigurations': [
        {
            'LambdaFunctionArn': lambda_function_arn,
            'Events': ['s3:ObjectCreated:*']
        }
    ]
}

# Save the notification configuration to a temporary JSON file
with open('notification_config.json', 'w') as json_file:
    json.dump(notification_config, json_file)

# Use AWS CLI to set the notification configuration
subprocess.run(['aws', 's3api', 'put-bucket-notification-configuration', 
                '--bucket', bucket_name,
                '--notification-configuration', f'file://notification_config.json'])

print('S3 event notification configured')


S3 event notification configured


In [ ]:
import boto3
import json

def create_s3_event_notification(bucket_name, lambda_function_arn):
    # Create an S3 client
    s3_client = boto3.client('s3')

    # Define the Lambda function configuration
    lambda_function_config = {
        'LambdaFunctionArn': lambda_function_arn,
        'Events': ['s3:ObjectCreated:*']
    }

    # Define the notification configuration
    notification_config = {
        'LambdaFunctionConfigurations': [lambda_function_config]
    }

    try:
        # Set the notification configuration for the S3 bucket
        s3_client.put_bucket_notification_configuration(
            Bucket=bucket_name,
            NotificationConfiguration=notification_config
        )

        print(f'S3 event notification configured for bucket: {bucket_name}')
    except Exception as e:
        print(f'Error configuring S3 event notification: {e}')

# Replace these variables with your own values
bucket_name = 'ieshaan-bucket-latest'
lambda_function_arn = 'arn:aws:lambda:us-west-2:935670829844:function:glue_dynamo_trigger'

# Create S3 event notification
create_s3_event_notification(bucket_name, lambda_function_arn)
